In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN, Dense,Input

In [2]:
max_features = 10000 # vocab size
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=max_features)


print(f'Training data shape:{X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape:{X_test.shape}, Testing labels shape:{y_test.shape}')
# usne test data ka toh kuch nhi kiya so i will be taking the charge to increase tha available data and epochs

Training data shape:(25000,), Training labels shape: (25000,)
Testing data shape:(25000,), Testing labels shape:(25000,)


In [3]:
# X=np.concatenate((X_train,X_test),axis = 0)
# y=np.concatenate((y_train,y_test),axis = 0)

# X_train,X_test,y_train,y_test = tts(X,y,test_size=0.2,random_state=42)   as we will be using cross validation ahead while training and all the data will be from same distribution.
X=np.concatenate((X_train,X_test),axis = 0)
y=np.concatenate((y_train,y_test),axis = 0)
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.1, random_state=43)

print(f'Training data shape:{X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape:{X_test.shape}, Testing labels shape:{y_test.shape}')

Training data shape:(45000,), Training labels shape: (45000,)
Testing data shape:(5000,), Testing labels shape:(5000,)


In [4]:
X_test[0] ## one hot rep

[1,
 2509,
 1162,
 7506,
 5,
 642,
 13,
 426,
 570,
 1104,
 149,
 14,
 22,
 5172,
 2,
 2423,
 4,
 102,
 29,
 93,
 23,
 4,
 223,
 5802,
 1790,
 56,
 11,
 2929,
 2642,
 11,
 4,
 5453,
 45,
 35,
 221,
 168,
 33,
 6,
 8178,
 1723,
 5,
 27,
 4484,
 5,
 1256,
 8,
 8803,
 18,
 4,
 370,
 7673,
 9,
 6,
 565,
 681,
 190,
 92,
 75,
 32,
 106,
 102,
 8,
 30,
 8282,
 11,
 49,
 96,
 42,
 160,
 10,
 10,
 39,
 4,
 454,
 13,
 16,
 5184,
 33,
 4,
 192,
 15,
 4,
 1013,
 11,
 4,
 108,
 468,
 8,
 977,
 2,
 1945,
 614,
 303,
 5172,
 795,
 44,
 27,
 8178,
 6250,
 5,
 25,
 70,
 67,
 12,
 39,
 4,
 4199,
 194,
 4199,
 7,
 2,
 33,
 35,
 402,
 559,
 279,
 12,
 16,
 43,
 4,
 2272,
 2929,
 2642,
 5825,
 37,
 694,
 17,
 6,
 992,
 132,
 37,
 82,
 2083,
 56,
 11,
 6,
 2,
 2776,
 45,
 87,
 8,
 67,
 15,
 27,
 846,
 5067,
 5,
 447,
 90,
 5,
 15,
 27,
 369,
 468,
 8,
 30,
 2,
 19,
 27,
 2,
 676,
 496,
 4,
 7315,
 10,
 10,
 276,
 4,
 370,
 23,
 72,
 1580,
 6,
 168,
 145,
 8,
 4,
 1355,
 2,
 34,
 2,
 490,
 377,
 4,
 1153,
 

In [5]:
print(f"Sample review in number:{X_train[0]} and its label:{y_train[0]}")
sample_review = X_train[0]

Sample review in number:[1, 48, 14, 22, 69, 6, 352, 7, 891, 1431, 474, 43, 40, 8, 124, 121, 4, 278, 435, 6, 3606, 100, 97, 128, 1683, 302, 95, 51, 16, 1053, 18, 342, 634, 23, 14, 2150, 418, 7, 1244, 261, 13, 215, 974, 4, 3869, 5, 4, 8254, 62, 28, 610, 66, 66, 646, 23, 35, 204, 297, 1709, 300, 500, 5, 402, 6090, 1628, 39, 4, 1696, 4773, 48, 6, 500, 69, 126, 77, 93, 51, 2, 72, 9, 121, 122, 4, 278, 140, 2, 108, 100, 28, 93, 6, 87, 22, 19, 52, 154, 3357, 4122, 5, 1222, 5407, 3965, 6, 1080, 742, 3651, 7962, 6592, 2, 5, 156, 37, 165, 168, 40, 36, 459, 44, 51, 36, 26, 399, 42, 7224, 11, 14, 420, 18, 44, 4, 172, 891, 1431, 14, 9, 179, 869, 4, 249, 22, 126, 93, 13, 62, 247, 870, 143, 6, 3333, 534, 3260, 2824, 7, 2, 74, 106, 14, 22, 174, 13, 1800, 12, 340, 13, 2, 13, 135, 14, 9, 4, 249, 22, 126, 93, 88, 1025, 85, 78, 102, 40, 1347, 790, 42, 455, 4808, 42, 2062, 5222, 2, 4, 8254, 134, 26, 108, 15, 26, 38, 78, 25, 28, 6, 318, 273, 11, 129, 483, 18, 98, 25, 119, 98, 50, 9, 57, 119, 18, 14, 22, 5, 5

In [6]:
word_index = imdb.get_word_index()

In [7]:
# word_index

In [8]:
rev_word_index = {value:key for key,value in word_index.items()}
# rev_word_index

In [9]:
decoded_review = ' '.join([rev_word_index.get(i - 3,'?') for i in sample_review])  # as in tf docs
decoded_review

"? if this film had a budget of 20 million i'd just like to know where the money went a monkey could make better cgi effects then what was wasted for 3 hours on this dreadful piece of garbage although i must admit the machines and the martians would have looked really really cool on an original play station 1 game and early pc games from the mid 90s if a game had ever been made what ? me is where did the money go ? films could have made a great film with good old fashioned models and computer controlled cameras a la george lucas circa 1975 ? and actors who actually look like they care about what they are doing or ruining in this case for about the same 20 million this is quite possibly the worst film ever made i would rather sit through a 24 hour repeat screening of ? than watch this film again i hated it completely i ? i say this is the worst film ever made because unlike other bad movies like plan 9 or killer tomatoes or santa claus ? the martians these are films that are so bad you 

In [10]:
maxlen = 500
X_train  = sequence.pad_sequences(X_train,maxlen=maxlen)
X_test  = sequence.pad_sequences(X_test,maxlen=maxlen)
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [11]:

# Train our simple rnn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, SimpleRNN, Embedding, Input,BatchNormalization
from tensorflow.keras import regularizers



def custom_activation(x):
    return (tf.nn.tanh(x) + 1) / 2

l2_reg = regularizers.l2(0.01)

dim = 128
model = Sequential()
model.add(Input((maxlen,)))
model.add(Embedding(max_features, dim))
model.add(SimpleRNN(128,kernel_regularizer=l2_reg))# 256 neurons in SimpleRNN
model.add(BatchNormalization())
model.add(LeakyReLU(negative_slope=0.1))  # LeakyReLU activation after SimpleRNN
model.add(Dense(64,kernel_regularizer=l2_reg))  # 64 neurons in Dense layer
model.add(BatchNormalization())
model.add(LeakyReLU(negative_slope=0.1))  # LeakyReLU activation after Dense
model.add(Dense(1, activation=custom_activation))  # Output layer for binary classification

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,321,985 (5.04 MB)

 Trainable params: 1,321,601 (5.04 MB)

 Non-trainable params: 384 (1.50 KB)

In [12]:
# Compile with lower learning rate and gradient clipping
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=1e-3, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# Create an instance of early stopping
from tensorflow.keras.callbacks import EarlyStopping
earlystopping  = EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)
earlystopping


In [14]:

# Train the model with early stopping 
history = model.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_split=0.1, 
    callbacks=[earlystopping])

Epoch 1/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 73s 229ms/step - accuracy: 0.5477 - loss: 2.2207 - val_accuracy: 0.4951 - val_loss: 1.5956
Epoch 2/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 73s 229ms/step - accuracy: 0.6260 - loss: 0.9157 - val_accuracy: 0.7702 - val_loss: 0.6241
Epoch 3/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 71s 225ms/step - accuracy: 0.8384 - loss: 0.4615 - val_accuracy: 0.8360 - val_loss: 0.4998
Epoch 4/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 73s 231ms/step - accuracy: 0.8757 - loss: 0.3384 - val_accuracy: 0.8460 - val_loss: 0.4425
Epoch 5/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 73s 230ms/step - accuracy: 0.8793 - loss: 0.3329 - val_accuracy: 0.8669 - val_loss: 0.4984
Epoch 6/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 74s 233ms/step - accuracy: 0.8919 - loss: 0.3115 - val_accuracy: 0.8604 - val_loss: 0.4062
Epoch 7/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 71s 225ms/step - accuracy: 0.9062 - loss: 0.2761 - val_accuracy: 0.8178 - val_loss: 0.4839
Epoch 8/30
317/317 ━━━━━━━━━━━━━━━━━━━━ 71s 225ms/step - accuracy: 0.9050 - loss: 0

In [15]:
from sklearn.metrics import accuracy_score

# Get model predictions
y_hat = model.predict(X_test)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_hat > 0.50).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step
Accuracy: 0.8598


In [16]:
model.save('simple_rnn_model.keras')

In [17]:
y_pred = (y_hat > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8598
